In [ ]:
# from google.colab import userdata (only in colab)

Libraries Installation

In [19]:
pip install -q PyPDF2 pinecone-client cohere
from IPython.display import Markdown
import textwrap
from dotenv import load_dotenv
import os
import cohere
from pinecone import Pinecone, ServerlessSpec
load_dotenv()

API keys Setup

In [ ]:
COHERE_API_KEY = os.getenv('COHERE_API_KEY')
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')

Cohere Setup and example Working

In [15]:
# Example with Cohere for embedding generation

co = cohere.Client(COHERE_API_KEY)
texts = ["Rag is retrieval augmented generation where response i sgenerated referencing the provided document", "machine learnig is divided in supervised, semi-supervised, reinforcement and unsupervise", "Lstm is good while dealing with longer context data while GRU is better for dealing with text classification"]  # your dataset
response = co.embed(texts=texts, model="embed-english-v2.0")
embeddings = response.embeddings  # List of embeddings

sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\hp\AppData\Local\sagemaker\sagemaker\config.yaml


PineCone setup and index creation as a database for storing mbeddings

In [11]:
# Initialize Pinecone
pc = Pinecone(api_key=PINECONE_API_KEY)

def create_index():
  pc.create_index(
      name="example-index",
      dimension=4096,
      metric="cosine",
      spec=ServerlessSpec(
          cloud='aws',
          region='us-east-1'
      )
  )

In [12]:
create_index()

Data ingestion in Pinecone

In [17]:
index = pc.Index("example-index")

# Upload embeddings to Pinecone
for i, embed in enumerate(embeddings):
    index.upsert([(str(i), embed)])  # Document IDs are i

Pipeline integration and working through example queries

In [ ]:
# Embed the query
query = ["How does Rag work?", "what are subsets of machine learning", "differenece in lstm and gru"]
query_embed = co.embed(texts=[query], model="embed-english-v2.0").embeddings[0]

# Retrieve the closest document embeddings
for i in query_embed:
    top_k = 5
    result = index.query(vector=i, top_k=top_k)
    retrieved_docs = [texts[int(match['id'])] for match in result['matches']]

    # Generate answer using Cohere or any generative model
    context = " ".join(retrieved_docs)

    for i in query:
        response = co.generate(prompt=f"{context}\n\nQuestion: {i}\nAnswer:", max_tokens=100)
        print(f"Response: {response.generations[0].text}")
        print(f"Retrieved texts: {retrieved_docs}")